# Extract, Transform, Load (ETL)

### 1. **Extração (Extract)**

A fase de extração envolve a coleta de dados de múltiplas fontes, que podem estar em diferentes formatos como CSV ou JSON. Para extrair os dados, utilizamos funções que lêem esses arquivos e os convertem em um formato manipulável, geralmente dataframes. A função glob é frequentemente utilizada para identificar e listar todos os arquivos com uma extensão específica em um diretório. Por exemplo, *.csv ou *.json retornaria todos os arquivos CSV ou JSON respectivamente.

As funções de extração são projetadas para lidar com diferentes formatos de arquivo: CSV, JSON e XML. Cada função lê os dados de um arquivo específico e os converte em um DataFrame do Pandas.

- **Extração de CSV**:
  ```python
  def extract_from_csv(file_to_process): 
      dataframe = pd.read_csv(file_to_process) 
      return dataframe 
  ```

- **Extração de JSON**:
  ```python
  def extract_from_json(file_to_process): 
      dataframe = pd.read_json(file_to_process, lines=True) 
      return dataframe 
  ```

- **Extração de XML**:
  ```python
  def extract_from_xml(file_to_process): 
      dataframe = pd.DataFrame(columns=["name", "height", "weight"]) 
      tree = ET.parse(file_to_process) 
      root = tree.getroot() 
      for person in root: 
          name = person.find("name").text 
          height = float(person.find("height").text) 
          weight = float(person.find("weight").text) 
          dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True) 
      return dataframe 
  ```

A função **`extract()`** orquestra a extração de todos os arquivos encontrados nos diretórios:

```python
def extract(): 
    extracted_data = pd.DataFrame(columns=['name','height','weight'])
    for csvfile in glob.glob("*.csv"): 
        extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True) 
    for jsonfile in glob.glob("*.json"): 
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True) 
    for xmlfile in glob.glob("*.xml"): 
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True) 
    return extracted_data 
```

### 2. **Transformação (Transform)**

Após a extração, os dados frequentemente precisam ser transformados para atender aos requisitos do sistema de destino ou para facilitar análises futuras. Isso pode incluir conversões de unidades (por exemplo, de polegadas para metros), limpeza de dados, ou mesmo agregações e cálculos específicos.

No exemplo dado, a altura em pés é convertida para metros e o peso em libras é convertido para quilogramas. Isso é feito aplicando operações matemáticas nos valores das colunas correspondentes nos dataframes.

A função **`transform(data)`** ajusta as unidades de medidas dos dados, convertendo altura de polegadas para metros e peso de libras para quilogramas:

```python
def transform(data): 
    data['height'] = round(data.height * 0.0254, 2)
    data['weight'] = round(data.weight * 0.45359237, 2)
    return data
```

### 3. **Carregamento (Load)**

A última etapa do processo ETL é carregar os dados transformados no sistema de destino. Isso pode ser um banco de dados, um arquivo CSV ou qualquer outro repositório de dados. No exemplo, o dataframe é salvo como um arquivo CSV utilizando a função to_csv do pandas.

Os dados transformados são salvos em um arquivo CSV. A função **`load_data()`** executa essa tarefa:

```python
def load_data(target_file, transformed_data): 
    transformed_data.to_csv(target_file)
```

### 4. **Registro de Log (Logging)**

Além das três principais funções de ETL, é essencial manter um registro das operações realizadas. Isso geralmente envolve salvar entradas de log com timestamps que indicam quando cada etapa do processo foi iniciada e concluída. No Python, isso pode ser feito utilizando o módulo datetime para capturar o tempo atual e escrever essa informação em um arquivo de log.

O processo de log é manipulado pela função **`log_progress(message)`**, que registra o progresso do processo ETL com carimbos de data/hora:

```python
def log_progress(message): 
    timestamp_format = '%Y-%h-%d-%H:%M:%S'  # Formato da data e hora
    now = datetime.now() 
    timestamp = now.strftime(timestamp_format) 
    with open(log_file, "a") as f: 
        f.write(timestamp + ',' + message + '\n') 
```

### Execução do Processo ETL e Registros de Log

O script segue uma ordem sequencial para chamar as funções de ETL e registrar cada etapa:

```python
log_progress("ETL Job Started")
log_progress("Extract phase Started")
extracted_data = extract()
log_progress("Extract phase Ended")

log_progress("Transform phase Started")
transformed_data = transform(extracted_data)
print("Transformed Data")
print(transformed_data)

log_progress("Transform phase Ended")
log_progress("Load phase Started")
load_data(target_file, transformed_data)
log_progress("Load phase Ended")

log_progress("ETL Job Ended")
```

Cada chamada para `log_progress()` assegura que você tenha um registro detalhado do progresso e possíveis pontos de falha, melhorando a transparência e a rastreabilidade do processo ETL.

# Web Scraping

### O que é Web Scraping?

Web scraping é a técnica de extrair dados de websites de forma programática. É útil para coletar dados de várias fontes onde não existe uma API disponível para extrair informações de forma mais direta. Imagine que você queira analisar os melhores jogadores de uma liga de basquete, web scraping pode te ajudar a coletar esses dados rapidamente.

### Introdução ao BeautifulSoup e Requests

Para começar, precisamos importar duas bibliotecas essenciais:

```python
from bs4 import BeautifulSoup
import requests
```

- **Requests**: Utilizada para fazer o download do conteúdo da página da web.
- **BeautifulSoup**: Utilizada para analisar e extrair informações do conteúdo HTML.

### Obtendo e Analisando uma Página Web

Vamos supor que você precise extrair o nome e salário dos jogadores de uma liga de basquete a partir de uma página web:

1. **Baixando a página**:
   ```python
   url = 'http://example.com'
   response = requests.get(url)
   page = response.text
   ```

2. **Criando um objeto BeautifulSoup**:
   ```python
   soup = BeautifulSoup(page, 'html.parser')
   ```

### Explorando o Objeto BeautifulSoup

O objeto `soup` criado permite que você navegue pela estrutura HTML da página como uma estrutura de dados aninhada. Aqui estão algumas maneiras de interagir com ele:

- **Acessando tags**: 
  ```python
  title_tag = soup.title
  h3_tag = soup.h3
  ```

- **Navegando pela árvore**:
  - **Acessando filhos**:
    ```python
    bold_tag = soup.b.find()
    ```
  - **Acessando o pai**:
    ```python
    parent_tag = bold_tag.parent
    ```
  - **Acessando irmãos**:
    ```python
    next_sibling = bold_tag.next_sibling
    ```

### Utilizando `find_all` para Filtrar Dados

`find_all()` é um método poderoso que busca por todas as instâncias de uma tag, com base em filtros que você especifica:

```python
rows = soup.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    for cell in cells:
        print(cell.text)
```

Esse trecho de código itera sobre todas as linhas e células de uma tabela, extraindo e imprimindo o texto de cada célula.

### Aplicando BeautifulSoup em um Site Real

Para fazer web scraping de um site real, você seguirá um processo similar:

1. **Importe as bibliotecas e baixe a página**.
2. **Crie o objeto BeautifulSoup**.
3. **Utilize métodos como `find()` e `find_all()` para extrair informações específicas**.

Por exemplo, para extrair dados de uma tabela:

```python
table = soup.find('table')  # Encontre a tabela
rows = table.find_all('tr')  # Extraia todas as linhas da tabela

for row in rows:
    cells = row.find_all('td')  # Encontre todas as células de cada linha
    data = [cell.text for cell in cells]  # Extraia o texto de cada célula
    print(data)  # Imprima os dados da célula
```

# REST APIs & HTTP Requests

Vamos explorar o uso da biblioteca Requests em Python, uma ferramenta poderosa para trabalhar com o protocolo HTTP. A biblioteca Requests torna muito mais fácil enviar requisições HTTP/1.1, sejam elas GET ou POST, entre outras. Ao entender e aplicar esta biblioteca, você poderá interagir com APIs da web ou automatizar a coleta de dados de páginas da internet de maneira eficaz.

### Importando a Biblioteca Requests

Primeiro, precisamos importar a biblioteca Requests:

```python
import requests
```

### Fazendo uma Requisição GET

Vamos começar com uma requisição GET, que é usada para solicitar dados de um servidor. É o tipo de requisição mais comum para recuperar informações de páginas da web ou APIs.

#### Exemplo de Requisição GET

Suponha que queremos pegar o conteúdo da página da IBM:

```python
response = requests.get('http://www.ibm.com')
```

Neste ponto, o objeto `response` contém muitas informações sobre a requisição feita:

- **Código de Status**:
  ```python
  print(response.status_code)  # 200 significa sucesso
  ```

- **Cabeçalhos da Resposta**:
  ```python
  print(response.headers)
  ```

  Isso retornará um dicionário dos cabeçalhos HTTP, onde você pode obter várias informações como a data da requisição ou o tipo de conteúdo respondido.

- **Conteúdo da Resposta**:
  Se o conteúdo da resposta for HTML, você pode acessá-lo com:

  ```python
  print(response.text[:100])  # Exibe os primeiros 100 caracteres do HTML
  ```

### Personalizando Requisições GET com Parâmetros

Você pode personalizar uma requisição GET para incluir parâmetros de consulta. Por exemplo, ao acessar uma API:

```python
params = {'name': 'Joseph', 'ID': '123'}
response = requests.get('http://httpbin.org/get', params=params)
print(response.url)  # Verifica a URL formada com os parâmetros de consulta
```

Essa URL conterá os parâmetros de consulta que especificamos no dicionário `params`.

### Fazendo uma Requisição POST

Diferentemente do GET, a requisição POST envia dados para o servidor e é comumente usada para enviar formulários ou fazer upload de arquivos.

#### Exemplo de Requisição POST

```python
payload = {'name': 'Joseph', 'ID': '123'}
response = requests.post('http://httpbin.org/post', data=payload)
print(response.text)
```

Neste exemplo, os dados enviados na requisição POST podem ser acessados no corpo da resposta, não na URL como na requisição GET.

### Diferenças Entre GET e POST

- **GET** é usado principalmente para solicitar dados do servidor e os parâmetros são enviados na URL.
- **POST** é usado para enviar dados ao servidor, enviar formulários, etc., e os dados são enviados no corpo da requisição, não na URL.

Ao usar o método `.json()` da resposta de uma requisição que retorna dados JSON, você pode convertê-los diretamente para um dicionário Python:

```python
response = requests.get('http://httpbin.org/get', params={'key': 'value'})
data = response.json()
print(data)
```

Este exemplo imprime a resposta JSON convertida em um dicionário Python.

# Hands-on: Web scraping and Extracting Data using APIs

Este script Python combina o uso das bibliotecas `requests`, `sqlite3`, `pandas` e `BeautifulSoup` para extrair informações sobre filmes de uma página da web, armazená-las em um DataFrame do pandas, exportar os dados para um arquivo CSV e, por fim, armazenar esses dados em um banco de dados SQLite. Vamos analisar cada parte do código em detalhes:

### Importação de Bibliotecas
```python
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup
```
- **requests**: Utilizada para fazer requisições HTTP a um servidor web.
- **sqlite3**: Usada para interagir com bancos de dados SQLite.
- **pandas**: Biblioteca para manipulação e análise de dados.
- **BeautifulSoup**: Facilita a extração de dados de arquivos HTML e XML.

### Definição de Variáveis Iniciais
```python
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
db_name = 'Movies.db'
table_name = 'Top_50'
csv_path = '/home/project/top_50_films.csv'
df = pd.DataFrame(columns=["Average Rank","Film","Year"])
count = 0
```
- **url**: Endereço da página da qual os dados serão extraídos.
- **db_name**: Nome do arquivo do banco de dados SQLite.
- **table_name**: Nome da tabela dentro do banco de dados onde os dados serão armazenados.
- **csv_path**: Caminho onde o arquivo CSV será salvo.
- **df**: DataFrame inicialmente vazio com colunas definidas para armazenar os dados.
- **count**: Contador para limitar o número de filmes a serem processados.

### Extração de Dados com BeautifulSoup
```python
html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')
```
- Aqui, `requests.get(url)` é usado para fazer uma requisição GET ao URL especificado e `.text` obtém o conteúdo da página em formato de texto. O resultado é então passado para `BeautifulSoup`, que parseia o HTML.

### Processamento dos Dados da Tabela
```python
tables = data.find_all('tbody')
rows = tables[0].find_all('tr')
```
- Extrai todas as instâncias de `<tbody>` do HTML parseado e, em seguida, seleciona a primeira tabela (`tables[0]`) para extrair todas as linhas (`<tr>`) dela.

### Loop para Extração e Armazenamento de Dados
```python
for row in rows:
    if count<50:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Average Rank": col[0].contents[0],
                         "Film": col[1].contents[0],
                         "Year": col[2].contents[0]}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)
            count+=1
    else:
        break
```
- O loop itera sobre cada linha da tabela para extrair células (`<td>`). Se existirem células, os dados são extraídos e um novo DataFrame (`df1`) é criado para cada linha. Este DataFrame é então concatenado ao DataFrame principal (`df`).
- O loop continua até que 50 filmes sejam adicionados ao DataFrame.

### Salvar o DataFrame em um Banco de Dados SQLite
```python
conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()
```
- Estabelece uma conexão com o banco de dados SQLite (`Movies.db`).
- Usa `df.to_sql` para salvar o DataFrame em uma tabela (`Top_50`) no banco de dados. Se a tabela já existir, ela será substituída.
- Fecha a conexão com o banco de dados.

In [2]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
db_name = 'Movies.db'
table_name = 'Top_50'
csv_path = '/home/project/top_50_films.csv'
df = pd.DataFrame(columns=["Average Rank","Film","Year"])
count = 0

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

for row in rows:
    if count<50:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Average Rank": col[0].contents[0],
                         "Film": col[1].contents[0],
                         "Year": col[2].contents[0]}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
            count+=1
    else:
        break

print(df)

#df.to_csv(csv_path)

conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

   Average Rank                                           Film  Year
0             1                                  The Godfather  1972
1             2                                   Citizen Kane  1941
2             3                                     Casablanca  1942
3             4                         The Godfather, Part II  1974
4             5                            Singin' in the Rain  1952
5             6                                         Psycho  1960
6             7                                    Rear Window  1954
7             8                                 Apocalypse Now  1979
8             9                          2001: A Space Odyssey  1968
9            10                                  Seven Samurai  1954
10           11                                        Vertigo  1958
11           12                                    Sunset Blvd  1950
12           13                                   Modern Times  1936
13           14                   

# Hands-on: Accessing Databases using Python script

### Importação de Bibliotecas e Conexão com o Banco de Dados

```python
import sqlite3
import pandas as pd
conn = sqlite3.connect('STAFF.db')
```
- **`sqlite3`**: Biblioteca usada para criar e gerenciar conexões com bancos de dados SQLite.
- **`pandas`**: Biblioteca para manipulação e análise de dados.
- A conexão com o banco de dados SQLite chamado `STAFF.db` é estabelecida.

### Leitura de Dados e Criação de Tabela

```python
table_name = 'INSTRUCTOR'
attribute_list = ['ID', 'FNAME', 'LNAME', 'CITY', 'CCODE']
file_path = '/home/project/INSTRUCTOR.csv'
df = pd.read_csv(file_path, names = attribute_list)
df.to_sql(table_name, conn, if_exists = 'replace', index =False)
print('Table is ready')
```
- **Definição das colunas**: As colunas para o DataFrame são definidas.
- **Leitura do arquivo CSV**: O DataFrame `df` é criado lendo os dados do arquivo `INSTRUCTOR.csv`, onde o parâmetro `names` especifica os nomes das colunas.
- **Gravação no SQLite**: O DataFrame é escrito na tabela `INSTRUCTOR` no banco de dados SQLite. Se a tabela já existir, ela é substituída.
- **Confirmação de criação da tabela**: Uma mensagem "Table is ready" é exibida.

### Execução e Impressão de Consultas SQL

```python
query_statement = f"SELECT * FROM {table_name}"
query_output = pd.read_sql(query_statement, conn)
print(query_statement)
print(query_output)

query_statement = f"SELECT FNAME FROM {table_name}"
query_output = pd.read_sql(query_statement, conn)
print(query_statement)
print(query_output)

query_statement = f"SELECT COUNT(*) FROM {table_name}"
query_output = pd.read_sql(query_statement, conn)
print(query_statement)
print(query_output)
```
- **Seleção de dados**: Diferentes consultas SQL são executadas para selecionar e exibir dados da tabela `INSTRUCTOR`. A primeira consulta seleciona todos os dados, a segunda apenas os nomes e a terceira conta o número de registros.
- **`pd.read_sql`**: A função `read_sql` do pandas executa a consulta SQL e retorna os resultados como um DataFrame.

### Adição de Novos Dados

```python
data_dict = {'ID' : [100],
             'FNAME' : ['John'],
             'LNAME' : ['Doe'],
             'CITY' : ['Paris'],
             'CCODE' : ['FR']}
data_append = pd.DataFrame(data_dict)
data_append.to_sql(table_name, conn, if_exists = 'append', index =False)
print('Data appended successfully')
```
- **Criação de novos dados**: Um novo registro é definido como um dicionário e convertido em um DataFrame.
- **Adição ao banco de dados**: O novo registro é adicionado à tabela `INSTRUCTOR` usando o modo `append`, que adiciona os dados sem substituir os existentes.
- **Confirmação de adição de dados**: Uma mensagem "Data appended successfully" é exibida.

### Fechamento da Conexão

```python
conn.close()
```
- **Encerramento da conexão**: A conexão com o banco de dados é fechada para liberar recursos.

# Practice Project: Extract, Transform and Load GDP data

Este script é um exemplo clássico de um processo ETL (Extract, Transform, Load) usando Python para manipular e armazenar dados de PIB de diferentes países obtidos de uma página da Wikipedia. Ele utiliza bibliotecas como `BeautifulSoup` para scraping web, `pandas` para manipulação de dados, `sqlite3` para operações de banco de dados, e implementa funções customizadas para cada etapa do processo ETL.

### Importação de Bibliotecas e Definição de Variáveis Globais

```python
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 
```

- **BeautifulSoup**: Utilizado para fazer parsing do HTML e extrair dados.
- **requests**: Para fazer requisições HTTP.
- **pandas e numpy**: Para manipulação de dados.
- **sqlite3**: Para operações de banco de dados.
- **datetime**: Para registrar logs com timestamps.

### Definição de Variáveis de Configuração
```python
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'
```
Essas variáveis armazenam as informações necessárias para a localização dos dados na web, bem como os caminhos para salvar os dados extraídos.

### Função `extract`

```python
def extract(url, table_attribs):
    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)

    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')

    for row in rows:
        col = row.find_all('td')
        if len(col) != 0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df, df1], ignore_index=True)

    return df
```
- A função inicia fazendo uma requisição HTTP para obter a página HTML.
- Usa `BeautifulSoup` para parsear o HTML e localizar as tabelas dentro da página. O script então foca na terceira tabela (`tables[2]`), assumindo que esta contém os dados de interesse.
- Itera sobre cada linha (`tr`) da tabela, e para cada linha, extrai as células (`td`). Se a célula contém um link (`a`) e o valor do PIB não é um traço (indicando dados ausentes), os dados são extraídos e adicionados a um DataFrame temporário `df1`, que é então concatenado ao DataFrame principal `df`.

### Função `transform`

```python
def transform(df):
    df['GDP_USD_millions'] = df['GDP_USD_millions'].replace('[\$,]', '', regex=True).astype(float)
    df['GDP_USD_millions'] = round(df['GDP_USD_millions'] / 1000, 2)
    df.rename(columns={'GDP_USD_millions': 'GDP_USD_billions'}, inplace=True)
    return df
```
- Esta função primeiro limpa e converte os valores de PIB, que estão em formato de texto com símbolos de moeda e vírgulas, para um tipo numérico (float).
- Em seguida, converte os valores de milhões para bilhões e arredonda para duas casas decimais.
- Renomeia a coluna para refletir que os valores agora estão em bilhões.

### Função `load_to_csv`

```python
def load_to_csv(df, csv_path):
    df.to_csv(csv_path, index=False)
```
- Simplesmente salva o DataFrame transformado como um arquivo CSV no caminho especificado.

### Função `load_to_db`

```python
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
```
- Carrega o DataFrame transformado em uma tabela no banco de dados SQLite especificado. Se a tabela já existir, ela será substituída.

### Função `run_query`

```python
def run_query(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)
```
- Executa uma consulta SQL utilizando a conexão ao banco de dados especificada e imprime tanto a consulta quanto o resultado.

### Função `log_progress`

```python
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./etl_project_log.txt", "a") as f:
        f.write(timestamp + ',' + message + '\n')
```
- Registra uma mensagem de log com timestamp no arquivo `etl_project_log.txt`. Este log ajuda a rastrear o progresso do processo ETL.

### Execução Principal

```python
log_progress('Preliminaries complete. Initiating ETL process')
df = extract(url, table_attribs)

log_progress('Data extraction complete. Initiating Transformation process')
df = transform(df)

log_progress('Data transformation complete. Initiating loading process')
load_to_csv(df, csv_path)

log_progress('Data saved to CSV file')
sql_connection = sqlite3.connect('World_Economies.db')
log_progress('SQL Connection initiated.')

load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as table. Running the query')

query_statement = f"SELECT * from {table_name} WHERE GDP_USD_billions >= 100"
run_query(query_statement, sql_connection)

log_progress('Process Complete.')
sql_connection.close()
```
- Este bloco de código controla a execução de todo o processo ETL, chamando as funções definidas anteriormente e registrando cada etapa. Inicia com a extração dos dados, passa pela transformação, salva os dados em um CSV, carrega no banco de dados, executa uma consulta para verificar os dados e, finalmente, fecha a conexão com o banco de dados.

# Python Style Guide and Coding Practices

### Importância de Escrever Código Legível
Escrever código legível é essencial porque garante que sua equipe possa entender, manter e modificar facilmente o código-base. Um código legível reduz a complexidade de entender o que o código faz, tornando a depuração e os aprimoramentos mais gerenciáveis. Isso é especialmente crucial em ambientes colaborativos onde vários desenvolvedores trabalham no mesmo código.

### Convenções de Codificação em Python: PEP8
PEP8, ou Proposta de Melhoria do Python número 8, é um documento lançado pelo Python.org que detalha como formatar o código Python para máxima legibilidade. Aqui estão algumas diretrizes-chave do PEP8:

1. **Usar Espaços em Vez de Tabs para Indentação**: 
   - Por quê? Diferentes IDEs e editores de texto podem interpretar tabs de maneira diferente (por exemplo, alguns podem tratar um tab como três espaços, enquanto outros como quatro). Essa inconsistência pode levar a erros de formatação. Usar espaços garante que o código pareça o mesmo em qualquer editor.
   - O PEP8 especifica o uso de quatro espaços por nível de indentação para manter o código visualmente estruturado e legível.

2. **Linhas em Branco**:
   - Use linhas em branco para separar funções e classes para delinear claramente diferentes seções do código. Por exemplo, garantir que haja uma linha em branco entre o final de uma função e o início de uma definição de classe ajuda a identificar onde um bloco termina e outro começa.

3. **Espaços ao Redor de Operadores e Após Vírgulas**:
   - Inclua espaços ao redor de operadores e após vírgulas para tornar o código mais legível. Por exemplo, escrever `a = b + c` em vez de `a=b+c` torna mais fácil distinguir entre variáveis e operadores.

### Convenções de Codificação para Consistência e Gerenciabilidade
Para manter um código-base consistente e gerenciável, adira a estas práticas:

1. **Decomposição Funcional**:
   - Divida grandes blocos de código em funções menores. Isso não apenas realça a legibilidade do código, mas também melhora a reutilização e os testes. Por exemplo, defina uma função para tarefas repetitivas e chame-a quando necessário, em vez de reescrever o código várias vezes.

2. **Convenções de Nomes**:
   - **Funções e Arquivos**: Use letras minúsculas com sublinhados (por exemplo, `calcular_area` em vez de `CalcularArea`). Este estilo é consistente com as convenções de nomenclatura internas e de bibliotecas do Python.
   - **Classes**: Use CamelCase (por exemplo, `ProcessadorDeDados`), o que ajuda a diferenciar classes de funções.
   - **Constantes**: Use letras maiúsculas com sublinhados separando palavras (por exemplo, `MAX_TENTATIVAS`). Essa convenção sinaliza que o valor não deve mudar.

3. **Evitando Sublinhados em Nomes de Pacotes**:
   - Embora funções e variáveis usem sublinhados, nomes de pacotes geralmente não os utilizam (por exemplo, `meupacote` em vez de `meu_pacote`). Isso é para garantir compatibilidade e aderir às convenções de empacotamento do Python.

### Análise de Código Estática
A análise de código estática envolve verificar o código quanto a erros, violações de estilo e potenciais bugs sem executar o programa. É uma etapa crucial para manter a qualidade do código e a conformidade com padrões de codificação como o PEP8.

- **Ferramentas**: Use ferramentas como PyLint para analisar seu código. PyLint verifica as diretrizes do PEP8 e outros padrões de codificação comuns para ajudá-lo a identificar áreas que precisam de melhoria.

Em conclusão, adotar as diretrizes do PEP8 e práticas de codificação consistentes aumenta a legibilidade e a gerenciabilidade do seu código. Usar ferramentas de análise de código estática ajuda ainda mais nesses esforços, garantindo a conformidade com os padrões de codificação estabelecidos. Essa abordagem beneficia não apenas os desenvolvedores individuais, mas também as equipes, tornando o código-base mais acessível e fácil de gerenciar.

# Unit Testing

### O que é Teste Unitário?

Teste unitário é um método para validar se unidades de código estão funcionando conforme projetado. Uma unidade é uma parte menor e testável de uma aplicação. Aqui está um exemplo de uma unidade, que possui duas funções, `square` e `doubler`, no arquivo `mymodule.py`. A função `square` é escrita como `def square(numero): return numero ** 2`. Similarmente, o código para a função `doubler` é escrito como `def doubler(numero): return numero * 2`.

### Processo de Teste Unitário

Durante o desenvolvimento de código, você testará cada unidade. O teste é realizado em duas fases:

1. **Teste Local**: Primeiramente, você testará a unidade em seu sistema local. Se o teste falhar, você determinará a razão da falha e corrigirá o problema. Em seguida, você testará a unidade novamente.

2. **Teste no Servidor**: Após o teste unitário local passar, você precisará testar a unidade em um ambiente de servidor, como um servidor de teste de integração contínua e entrega contínua (CI/CD). Se a unidade falhar no teste do servidor, você receberá os detalhes da falha, deverá determinar e corrigir o problema. Uma vez que a unidade passe no teste do servidor, ela é integrada à base de código final.

### Construindo Testes Unitários

Vamos revisar algumas funções de teste para entender como construir testes unitários:

1. **Importar a Biblioteca Unittest**:
   ```python
   import unittest
   ```

2. **Importar Funções para Testar**:
   ```python
   from mymodule import square, doubler
   ```

3. **Construir a Classe de Teste Unitário**:
   ```python
   class TestMyModule(unittest.TestCase):
       def test_square(self):
           self.assertEqual(square(2), 4)

       def test_doubler(self):
           self.assertEqual(doubler(2), 4)
   ```

   - A classe `TestMyModule` herda de `unittest.TestCase`.
   - Métodos que começam com `test_` são reconhecidos automaticamente pelo framework de teste para serem executados.
   - `assertEqual()` é usado para verificar se o valor retornado pela função é igual ao esperado.

### Executando e Revisando o Teste

Após construir os testes, você os executará, e o output mostrará os resultados, indicando se os testes passaram ou falharam. Por exemplo:

- **Output de Sucesso**: Mostra que todos os testes passaram, indicando que as funções foram implementadas corretamente.

- **Output de Falha**: Mostra onde os testes falharam, permitindo que você veja qual teste não passou e por quê. Por exemplo, se a função `square` estava calculando o cubo em vez do quadrado, o teste falharia.

### Módulo, Pacote e Biblioteca em Python

1. **Módulo Python**:
   - Um módulo é um arquivo `.py` contendo definições e declarações de Python, que incluem funções, classes e variáveis. Você pode importar um módulo para outros scripts e notebooks. Por exemplo, se você tem um módulo chamado `module.py` com funções para calcular o quadrado e o dobro de um número, você pode importá-lo e usar essas funções em seu código.

2. **Pacote Python**:
   - Um pacote é uma coleção de módulos Python organizada em um diretório que inclui um arquivo `__init__.py`. Este arquivo diferencia o pacote de um diretório comum de scripts Python. Por exemplo, um pacote chamado `myproject` pode conter vários módulos e o arquivo `__init__.py`, que organiza esses módulos como um pacote.

3. **Biblioteca Python**:
   - Uma biblioteca é uma coleção de pacotes ou pode ser um único pacote com funcionalidades amplas. Exemplos de bibliotecas incluem NumPy, PyTorch e Pandas. Estes termos, pacote e biblioteca, são frequentemente usados de forma intercambiável.

### Criando um Pacote Python

Para criar um pacote Python, siga estes passos:

1. **Crie uma Pasta**:
   - Nomeie a pasta com o nome do pacote, por exemplo, `myproject`.

2. **Crie o Arquivo `__init__.py`**:
   - Este arquivo deve ser criado dentro da pasta do pacote e pode ser inicialmente um arquivo vazio ou incluir comandos de importação dos módulos dentro do pacote.

3. **Adicione Módulos**:
   - Crie os módulos necessários, como `module1.py` e `module2.py`, dentro da pasta do pacote. Esses módulos podem conter funções específicas como `square`, `doubler` e `mean`.

4. **Configure o Arquivo `__init__.py`**:
   - No arquivo `__init__.py`, adicione código para referenciar os módulos necessários dentro do pacote, por exemplo:
     ```python
     from .module1 import square, doubler
     from .module2 import mean
     ```

### Verificando um Pacote Python

Para verificar se o pacote está funcionando corretamente:

1. **Abra um Terminal Bash**:
   - Navegue até o diretório onde seu pacote está localizado.

2. **Inicie o Interpretador Python**:
   - Execute o comando `python` para abrir o interpretador.

3. **Importe o Pacote**:
   - Digite `import myproject`. Se o comando for executado sem erros, seu pacote foi carregado com sucesso.

### Usando um Pacote Python

Para usar o pacote em outros scripts, certifique-se de que a pasta do pacote esteja no mesmo diretório do script ou no caminho de pesquisa do Python. Importe as funções necessárias do pacote e use-as como desejado. Por exemplo:

```python
from myproject.module1 import square, doubler
from myproject.module2 import mean

print(square(4))  # Saída esperada: 16
print(doubler(4))  # Saída esperada: 8
print(mean([2, 1, 3]))  # Saída esperada: 2.0
```